In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
import numpy as np
import os
import time

2023-03-11 04:57:21.197451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

In [3]:
resnet_model = ResNet50(weights='imagenet')

2023-03-11 04:57:24.387755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 04:57:24.390175: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


In [4]:
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [6]:
# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
# Convert labels to one-hot vectors
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [8]:
# Define ResNet50 model with custom output layer
input_tensor = Input(shape=(32, 32, 3))
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)
x = GlobalAveragePooling2D()(base_model.output)
output_tensor = Dense(10, activation='softmax')(x)
model = Model(inputs=input_tensor, outputs=output_tensor)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
start_time = time.time()
model.fit(x_train, y_train, batch_size=1500, epochs=2, validation_data=(x_test, y_test))
end_time = time.time()

Epoch 1/2
34/34 [==============================] - 218s 6s/step - loss: 1.2603 - accuracy: 0.6012 - val_loss: 23.2662 - val_accuracy: 0.1000
Epoch 2/2
34/34 [==============================] - 199s 6s/step - loss: 0.5548 - accuracy: 0.8103 - val_loss: 7.9638 - val_accuracy: 0.1000


In [11]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 24s 77ms/step - loss: 7.9638 - accuracy: 0.1000


In [12]:
# Print the test loss, accuracy, and training time
print("Test loss:", loss)
print("Test accuracy:", acc)
print("Training time:", end_time - start_time, "seconds")

Test loss: 7.96384859085083
Test accuracy: 0.10000000149011612
Training time: 454.91341042518616 seconds


In [13]:
import matplotlib.pyplo
# Plot some sample images from the CIFAR-10 dataset
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i])
    ax.set_title(f"Label: {np.argmax(y_train[i])}")
plt.show()

NameError: name 'plt' is not defined

In [ ]:

# Plot the training and validation accuracy and loss
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Training and Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Training and Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

plt.show()